# Hands-on: Getting Started with Generative AI APIs
### by Mike Wolfson
### **Phoenix Mobile & Emerging Technology** - 2/15/24
### **DevFest Vancouver** - 11/18/2023
### **DevFest Waterloo** - 10/21/2023

In this hands-on workshop, participants will learn how to develop generative AI projects, harnessing the power of Jupyter Notebook. We'll demystify the foundations of generative AI, and get our hands dirty with real-time experiments. Whether you're a budding data scientist, or just intrigued by this emerging technology, join us as we unravel the potential of generative AI, one API call at a time. Prepare to leave with the knowledge, experience, and tools to craft your own generative masterpieces!



## Python

```
Python's simplicity, extensive libraries, community support, and versatility make it an ideal choice for AI development. It provides the tools and resources necessary to build AI models, work with data, and deploy AI solutions in a wide range of applications.
```

Download and Install Python [Site](https://www.python.org/downloads/)

Official Documentation [Site](https://docs.python.org/3/)

## PIP

**Pythons Package Manager** is now installed 

*We will be using this to download and install all of our project dependencies*

## Jupyter Notebook

- Introduction to Jupyter Notebooks.
- Benefits of using Jupyter for interactive development.
- Understanding cells, outputs, and extensions.

[Jupyter Documentation](https://docs.jupyter.org/en/latest/)


### PIP Install

 `pip install jupyter`

#### Once you have this package installed, navigate to the location on your file-system where you want to store your notebook file, and then enter the following command on the command-line to start the jupyter environment

`jupyter`

## Hugging Face Transformers API

PIP Install: 

`pip install transformers torch`

Documentation: [Install Instructions](https://huggingface.co/docs/transformers)

## Note

> This Step is intended to validate the successful installation of Python and jupyter **without requiring an API key**, which is needed in the next steps.

**You may want to skip this step** beacuse it downloads a large binary to your machine. If you intend to continue this workshop, you can skip directly to the "Working with Google APIs" section, where you will validate these your *python* and *jupyter* tool installations anyway.

### generate_text_huggingface: helper function

Use this to execute calls to the Hugging Face Transformers API

*No API Key needed!*

In [ ]:
from transformers import GPT2LMHeadModel, GPT2Tokenizer

def generate_text_huggingface(prompt, model_name="gpt2-medium"):
    tokenizer = GPT2Tokenizer.from_pretrained(model_name)
    model = GPT2LMHeadModel.from_pretrained(model_name)

    input_ids = tokenizer.encode(prompt, return_tensors='pt')
    output = model.generate(input_ids, max_length=150, num_return_sequences=1, temperature=1.0)
    
    generated_text = tokenizer.decode(output[0], skip_special_tokens=True)
    return generated_text

## Sending a prompt to the **Hugging Face Helper Function**

Note: You will see an error when running this cell.  You can safely ignore:

> The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.

As long as you see that the output shows "Once upon a time..." with a lot more text, then this step is complete.

In [ ]:
print(generate_text_huggingface("Once upon a time"))

## Working with Google APIs

```
Large Language Models (LLMs) are a powerful, versatile type of machine learning model that enables computers to comprehend and generate natural language better than ever. They can be used to build all sorts of applications, from chat bots to virtual assistants to translation apps and much more. Plus, you don't have to be an AI expert or even write code to use them. All it takes are a few sentences or “prompts” to get started designing your own custom LLM app.
```

### Use **PIP** to Install Dependencies 

- Google GenerativeAI Python Library - `pip install -q google.generativeai`
- Tools to grab API key from Environment variable: `pip install python-dotenv`

## Enable Gemini Services and get API Key

Get an API key by going here: [https://ai.google.dev/](https://ai.google.dev/)

Note: 
> If you have an existing Google Cloud API key, you can use it, but you will need to enable to the API in the Google Cloud Console.

### Working with API keys

You will need to set the Gemini API key as a system variable named: `GOOGLE_API_KEY`.  

- [Setting an Environment Variable on Mac/Linux](https://phoenixnap.com/kb/set-environment-variable-mac)
- [Setting an Environment Variable on Windows](https://phoenixnap.com/kb/windows-set-environment-variable)

#### Import GenerativeAI library

In [ ]:
import google.generativeai as genai

In [ ]:
import os
from dotenv import load_dotenv, find_dotenv

_ = load_dotenv(find_dotenv()) # read local .env file
apiKey = os.getenv('GOOGLE_API_KEY')

genai.configure(api_key=apiKey,
               transport="rest",
    )

## Explore the Available Models

Validate the library has been imported correctly by exploring available models.

> Successful output from this step validates successful setup of Google Generative AI library

In [ ]:
for m in genai.list_models():
    print(f"name: {m.name}")
    print(f"description: {m.description}")
    print(f"generation methods:{m.supported_generation_methods}\n")

### Filter models to ensure model we want is supported
- `generateContent` is the value we are looking for

In [ ]:
for m in genai.list_models():
  if 'generateContent' in m.supported_generation_methods:
    print(m.name)

Get a reference to the model

In [ ]:
model = genai.GenerativeModel('gemini-pro')

### generate_text_gemini: helper function to generate text

- The `@retry` decorator helps you to retry the API call if it fails.

In [ ]:
from google.api_core import retry
@retry.Retry()
def generate_text_gemini(prompt):
    response = model.generate_content(prompt)
    return response.text

## Sending a prompt to the **Google GenAI Helper Function**

In [ ]:
completion = generate_text_gemini("Thursday evenings are perfect for")
print(completion)

## Working with the Open AI APIs

```
OpenAI's APIs offer developers the ability to integrate advanced artificial intelligence capabilities into their applications, enabling a wide range of tasks from text generation to complex problem-solving.
```
Documentation: [https://beta.openai.com/docs/](https://beta.openai.com/docs/)

### Obtaining API Keys:
- **OpenAI Platform**: [https://platform.openai.com/](https://platform.openai.com/)
  - After signing up or logging in, navigate to the API section to manage and obtain your API keys.
- You will need to set the OpenAI API key as a system variable named: `OPENAI_API_KEY`.  

Note: do NOT check your API key into a public Github repo, or it will get revoked 
  
  


### PIP Dependencies

`pip install --upgrade openai`

In [ ]:
import openai
import os

from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv())

openai.api_key  = os.getenv('OPENAI_API_KEY')

### generate_text_openai: helper function to generate text from Open AI API

In [ ]:
from openai import OpenAI
client = OpenAI()

def generate_text_openai(pre, prompt, model="gpt-3.5-turbo"):
    completion = client.chat.completions.create(
    model=model,
    messages=[
        {"role": "system", "content": pre},
        {"role": "user", "content": prompt}
    ]
    )

    return completion.choices[0].message.content

## Sending a prompt to the **Open AI Helper Function**

In [ ]:
print(generate_text_openai("", "Thursday evenings are perfect for"))


## Prompt template

Using this simple template will help you organize your Prompts more effectively.

- **system**: establishing context about what the quesiton is about ("you are a pirate")
- **user**: what we are asking the GenAI to do ("you are telling us your favorite joke about booty")
- **format**: formatting and other after-thoughts ("use kid friendly language, and output the results in a json format")

I learned this prompt technique in the course "[Pair Programming with a Large Language Model](https://www.deeplearning.ai/short-courses/pair-programming-llm/)" - this is a short course by Laurence Moroney, and is a free course from [Deeplearning.ai](https://www.deeplearning.ai/)

In [ ]:
prompt_template = """
{user}

{format}
"""

In [ ]:
system = """
You are a software engineer and you are at the Phoenix AI Developers meetup. You are excited because generative AI is so interesting.
"""


In [ ]:
user = """
tell me about the most interesting ways to get started developing using generative AI APIs
"""

In [ ]:
format = """
mention the location and use flowery and descriptive language, and output the results as properly formatted markdown
"""

# format = """
# make sure to mention the location where you are and speak like a pirate, and output the results in a json format
# """

# format = """
# use child friendly language, and output the results in a json format
# """



In [ ]:
prompt = prompt_template.format(user=user, format=format)
print(generate_text_openai(system, prompt))